In [263]:
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm
import folium


In [264]:
def haversine_np(lon1, lat1, lon2, lat2, factor=1.25):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    # 1.25 because the road distance is, on average, 25% larger than a straight flight
    return factor * km

In [265]:
points = pd.read_csv("points.csv")
df = points[~points["dest_lon"].isna()]
df.head()

/var/folders/x9/x6m_v2y54yx2wmzg6z8dczgr0000gn/T/ipykernel_10630/433528718.py:1: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  points = pd.read_csv("points.csv")


,id,lat,lon,rating,country,wait,nickname,comment,datetime,reviewed,banned,ip,dest_lat,dest_lon,signal,ride_datetime,user_id,from_hitchwiki
18195,18207,49.635165,5.970091,4.0,LU,NaN,NaN,Got a ride after 10 minutes withhout even both...,2017-09-03 20:40:31.000000,1,0,NaN,47.686400,6.494300,NaN,NaN,NaN,NaN
49943,78546373,50.576778,-111.871741,1.0,CA,120.0,DSLR,"Freeway with high speed traffic, no good spots...",2022-11-06 07:37:11.530351,0,0,NaN,50.576778,-114.059811,NaN,NaN,NaN,0.0
49944,183954465,46.286302,-81.775566,4.0,CA,10.0,DSLR,NaN,2022-11-06 06:49:38.987729,0,0,NaN,46.286302,-84.341174,NaN,NaN,NaN,0.0
49945,397517487,50.998767,-118.229634,3.0,CA,100.0,DSLR,"Traffic is low and lots of local traffic, but ...",2022-11-06 07:49:16.533041,0,0,NaN,50.998767,-117.816645,NaN,NaN,NaN,0.0
49946,419660441,51.263986,-115.935150,5.0,CA,8.0,DSLR,The transcanada was closed between lake Louise...,2022-11-06 07:46:31.586044,0,0,NaN,51.263986,-118.214006,NaN,NaN,NaN,0.0


In [266]:
min_lon = 4.2
max_lon = 5
min_lat = 51
max_lat = 53
df = df[df["lon"] > min_lon]
df = df[df["lon"] < max_lon]
df = df[df["lat"] < max_lat]
df = df[df["lat"] > min_lat]
df = df[df["dest_lon"] > min_lon]
df = df[df["dest_lon"] < max_lon]
df = df[df["dest_lat"] < max_lat]
df = df[df["dest_lat"] > min_lat]


In [267]:
g = nx.DiGraph()

In [268]:
for _, row in df.iterrows():
    dist = haversine_np(row.lon, row.lat, row.dest_lon, row.dest_lat)
    if dist > 100:
        g.add_edge(
            (row.lon, row.lat),
            (row.dest_lon, row.dest_lat),
            weight=dist,
        )

In [269]:
g.edges(data=True)

OutEdgeDataView([((4.641155004501344, 51.358058223372055), (4.916381835937501, 52.35798989363364), {'weight': np.float64(140.88925387475976)}), ((4.464193582534791, 51.26278594613045), (4.880676269531251, 52.36805320057393), {'weight': np.float64(157.638463436889)}), ((4.4445168972, 51.2078714778), (4.9219608306884775, 52.376175552387224), {'weight': np.float64(167.38785094683374)}), ((4.41653609275818, 51.2636637361032), (4.492979049682618, 52.14665732565445), {'weight': np.float64(122.82982252297877)})])

In [270]:
len(g.nodes)

8

In [271]:
m = folium.Map([50.7, 4.2], zoom_start=7)

In [272]:
points = list(g.nodes)
for p in points:
    folium.Marker(location=(p[1], p[0])).add_to(m)

for e in g.edges:
    folium.PolyLine(
        [[e[0][1], e[0][0]], [e[1][1], e[1][0]]],
        color="#FF0000",
        weight=5,
    ).add_to(m)

    folium.RegularPolygonMarker(location=[e[1][1], e[1][0]], fill_color='blue', number_of_sides=3, radius=10).add_to(m)

m

In [273]:
# make it fully connected with walking paths
for n1 in g.nodes:
    for n2 in g.nodes:
        if n1 != n2:
            dist = haversine_np(n1[0], n1[1], n2[0], n2[1])
            g.add_edge(
                n1,
                n2,
                weight=dist * 20,
            ) 

In [274]:
points = list(g.nodes)
for p in points:
    folium.Marker(location=(p[1], p[0])).add_to(m)

for e in g.edges:
    folium.PolyLine(
        [[e[0][1], e[0][0]], [e[1][1], e[1][0]]],
        color="#FF0000",
        weight=5,
    ).add_to(m)

    folium.RegularPolygonMarker(location=[e[1][1], e[1][0]], fill_color='blue', number_of_sides=3, radius=10).add_to(m)

m

In [275]:
def find_route(G, departure, arrival):
    a = len(G.edges)

    prev_nodes = list(G.nodes)
    for n in prev_nodes:
        dist = haversine_np(arrival[0], arrival[1], n[0], n[1])
        G.add_edge(
            n,
            arrival,
            weight=dist * 20,
        )

    prev_nodes = list(G.nodes)
    for n in prev_nodes:
        dist = haversine_np(departure[0], departure[1], n[0], n[1])
        G.add_edge(
            departure,
            n,
            weight=dist * 20,
        )

  

    print(len(G.edges) - a)

    return G, nx.dijkstra_path(G, departure, arrival), nx.dijkstra_path_length(G, departure, arrival)
    

In [276]:
amsterdam = (4.7, 52.6) # Amsterdam
antwerp = (4.4, 51.1) # Barcelona
g, route, length = find_route(g, antwerp, amsterdam)
route, length

17


([(4.4, 51.1), (4.7, 52.6)], np.float64(4198.863125878438))

In [277]:
points = list(g.nodes)
for p in points:
    folium.Marker(location=(p[1], p[0])).add_to(m)

for e in g.edges:
    folium.PolyLine(
        [[e[0][1], e[0][0]], [e[1][1], e[1][0]]],
        color="#FF0000",
        weight=5,
    ).add_to(m)

    folium.RegularPolygonMarker(location=[e[1][1], e[1][0]], fill_color='blue', number_of_sides=3, radius=10).add_to(m)

m